In [28]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import math, sys, warnings, logging
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=FutureWarning)

try:
    import yfinance as yf
except Exception:
    print("Please: pip install yfinance pandas numpy")
    sys.exit(1)

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("rsi_vo_backtest")

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    tickers: List[str]
    start_date: str = "2025-10-01"
    end_date:   str = "2025-11-01"
    interval:   str = "5m"

    capital_per_stock: float = 100_000.0
    intraday_leverage: float = 5.0

    sl_rupees: float = 2000.0    # total max loss per position (₹)
    tp_rupees: float = 4000.0    # total target per position (₹)

    # Trailing SL (pure trailing: if price +₹X, SL +₹X; if price -₹X, SL unchanged)
    enable_trailing_sl: bool = False
    trail_rupees: float = 10.0    # ₹ per share DISTANCE from the extreme (no division by qty)

    # Directions
    enable_longs: bool = True
    enable_shorts: bool = True

    # Indicator params & thresholds
    rsi_len: int = 14
    vo_fast: int = 5
    vo_slow: int = 10
    vo_support_thresh: float = -30.0   # "support" (VO <= this) for longs
    vo_resist_thresh: float = 30.0     # "resistance" (VO >= this) for shorts

    # Trades/day cap (per symbol)
    max_trades_per_day: int = 3

    # Trading session (IST)
    entry_start: str = "09:30"
    entry_end:   str = "14:45"
    squareoff_time: str = "15:00"

    timezone: str = "Asia/Kolkata"
    out_file: str = "trades.csv"

CFG = Config(
    tickers=['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS',
             'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BEL.NS', 'BHARTIARTL.NS',
             'CIPLA.NS', 'COALINDIA.NS', 'DRREDDY.NS', 'EICHERMOT.NS',
             'ETERNAL.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS',
             'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ITC.NS', 'INFY.NS',
             'INDIGO.NS', 'JSWSTEEL.NS', 'JIOFIN.NS', 'KOTAKBANK.NS', 'LT.NS',
             'M&M.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'NTPC.NS', 'NESTLEIND.NS',
             'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SHRIRAMFIN.NS',
             'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TMPV.NS',
             'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'TRENT.NS', 'ULTRACEMCO.NS', 'WIPRO.NS']
)

# =========================
# FEES (Groww NSE intraday)
# =========================
def groww_intraday_charges(buy_turnover: float, sell_turnover: float) -> Dict[str, float]:
    def brokerage(turnover):
        fee = min(20.0, 0.001 * turnover)
        return max(5.0, fee)  # floor ₹5
    bro_buy  = brokerage(buy_turnover)
    bro_sell = brokerage(sell_turnover)
    exch_buy  = 0.0000297 * buy_turnover
    exch_sell = 0.0000297 * sell_turnover
    sebi_buy  = 0.000001 * buy_turnover
    sebi_sell = 0.000001 * sell_turnover
    ipft_buy  = 0.000001 * buy_turnover
    ipft_sell = 0.000001 * sell_turnover
    gst_buy  = 0.18 * (bro_buy  + exch_buy  + sebi_buy  + ipft_buy)
    gst_sell = 0.18 * (bro_sell + exch_sell + sebi_sell + ipft_sell)
    stt_sell = 0.00025 * sell_turnover
    stamp_buy = 0.00003 * buy_turnover
    total = (bro_buy + bro_sell + exch_buy + exch_sell +
             sebi_buy + sebi_sell + ipft_buy + ipft_sell +
             gst_buy + gst_sell + stt_sell + stamp_buy)
    return {"total_charges": total}

# =========================
# DATA
# =========================
def fetch_5m_dataframe(ticker: str, start: str, end: str, interval: str, tz: str) -> pd.DataFrame:
    df = yf.download(
        ticker, start=start, end=end, interval=interval,
        auto_adjust=False, prepost=False, progress=False,
        multi_level_index=False, group_by="column",
    )
    if df.empty:
        return df
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([c for c in col if c]).strip() for col in df.columns.values]
    if df.index.tz is None:
        df = df.tz_localize("UTC").tz_convert(tz)
    else:
        df = df.tz_convert(tz)
    df.rename(columns=lambda c: c.title(), inplace=True)
    keep = [c for c in ["Open", "High", "Low", "Close", "Volume"] if c in df.columns]
    df = df[keep]
    df["Ticker"] = ticker
    return df

def session_filter(df: pd.DataFrame, date: pd.Timestamp, tz: str) -> pd.DataFrame:
    day = pd.Timestamp(date)
    start = pd.Timestamp(f"{day.date()} 09:15").tz_localize(tz)
    end   = pd.Timestamp(f"{day.date()} 15:30").tz_localize(tz)
    return df.loc[(df.index >= start) & (df.index <= end)].copy()

def _ts_on_day(daylike: pd.Timestamp, hhmm: str, tz: str) -> pd.Timestamp:
    d = pd.Timestamp(daylike).date().strftime("%Y-%m-%d")
    return pd.Timestamp(f"{d} {hhmm}").tz_localize(tz)

# =========================
# INDICATORS
# =========================
def rsi_wilder(close: pd.Series, length: int = 14) -> pd.Series:
    delta = close.diff()
    gain = (delta.clip(lower=0)).ewm(alpha=1/length, adjust=False).mean()
    loss = (-delta.clip(upper=0)).ewm(alpha=1/length, adjust=False).mean()
    rs = gain / (loss.replace(0, np.nan))
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(50.0)

def volume_oscillator(vol: pd.Series, fast: int = 5, slow: int = 20) -> pd.Series:
    ema_fast = vol.ewm(span=fast, adjust=False).mean()
    ema_slow = vol.ewm(span=slow, adjust=False).mean()
    vo = (ema_fast - ema_slow) / ema_slow.replace(0, np.nan) * 100.0
    return vo.fillna(0.0)

def add_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    out = df.copy()
    out["RSI"] = rsi_wilder(out["Close"], cfg.rsi_len)
    out["VO"]  = volume_oscillator(out["Volume"].astype(float), cfg.vo_fast, cfg.vo_slow)
    return out

# =========================
# SIGNAL SCAN
# =========================
def next_signal_after(day_df: pd.DataFrame, cfg: Config, after_ts: Optional[pd.Timestamp] = None
                      ) -> Tuple[Optional[str], Optional[pd.Timestamp], Optional[float]]:
    """
    First RSI+VO signal after 'after_ts' (or after entry_start if None).
    Entry at NEXT bar OPEN of the signal bar.
    """
    if after_ts is not None:
        scan = day_df.loc[day_df.index > after_ts]
    else:
        start_ts = _ts_on_day(day_df.index[0], cfg.entry_start, cfg.timezone)
        scan = day_df.loc[day_df.index >= start_ts]
    end_ts = _ts_on_day(day_df.index[0], cfg.entry_end, cfg.timezone)
    scan = scan.loc[scan.index <= end_ts]
    if scan.empty:
        return None, None, None

    for ts, row in scan.iterrows():
        # Long: RSI < 30 and VO <= support
        if cfg.enable_longs and (row["RSI"] <= 30.0) and (row["VO"] <= cfg.vo_support_thresh):
            nxt = day_df.loc[day_df.index > ts]
            if not nxt.empty:
                return "long", nxt.index[0], float(nxt.iloc[0]["Open"])
        # Short: RSI > 70 and VO >= resistance
        if cfg.enable_shorts and (row["RSI"] >= 70.0) and (row["VO"] >= cfg.vo_resist_thresh):
            nxt = day_df.loc[day_df.index > ts]
            if not nxt.empty:
                return "short", nxt.index[0], float(nxt.iloc[0]["Open"])
    return None, None, None

# =========================
# SIMULATION (with pure trailing)
# =========================
def simulate_trade(day_df: pd.DataFrame, direction: str, entry_ts: pd.Timestamp, entry_price: float,
                   cfg: Config, ticker: str) -> Optional[Dict]:
    if entry_ts is None or math.isnan(entry_price):
        return None

    # Position sizing with intraday leverage
    buying_power = cfg.capital_per_stock * cfg.intraday_leverage
    qty = int(buying_power // entry_price)
    if qty < 1:
        return None

    # Convert position-level SL/TP to per-share distances
    risk_per_share   = cfg.sl_rupees / qty
    target_per_share = cfg.tp_rupees / qty
    trail_per_share  = cfg.trail_rupees  # <- pure trailing uses per-share rupees directly

    # Initial static SL and TP
    if direction == "long":
        static_sl = entry_price - risk_per_share
        tp = entry_price + target_per_share
        dyn_sl = static_sl
    else:
        static_sl = entry_price + risk_per_share
        tp = entry_price - target_per_share
        dyn_sl = static_sl

    exit_reason = None
    exit_ts = None
    exit_price = None

    after = day_df.loc[day_df.index >= entry_ts]

    for ts, row in after.iterrows():
        high = float(row["High"])
        low  = float(row["Low"])
        close = float(row["Close"])

        # --- True trailing-stop logic (₹ trail_per_share per share) ---
        if cfg.enable_trailing_sl:
            if direction == "long":
                # trail follows the *highest* tradeable price of this bar minus fixed ₹ distance
                new_sl = high - trail_per_share
                # never loosen below static or previously-best dyn_sl
                dyn_sl = max(dyn_sl, new_sl, static_sl)
            else:
                # trail follows the *lowest* tradeable price of this bar plus fixed ₹ distance
                new_sl = low + trail_per_share
                dyn_sl = min(dyn_sl, new_sl, static_sl)

        # --- Exit checks (adverse first is conservative) ---
        if direction == "long":
            # stop
            if low <= dyn_sl:
                exit_reason, exit_ts, exit_price = ("SL-TRAIL" if cfg.enable_trailing_sl else "SL", ts, dyn_sl)
                break
            # target
            if high >= tp:
                exit_reason, exit_ts, exit_price = ("TP", ts, tp)
                break
            # strategy exit: RSI>70
            if row["RSI"] >= 70.0:
                exit_reason, exit_ts, exit_price = ("RSI-EXIT", ts, close)
                break
        else:
            if high >= dyn_sl:
                exit_reason, exit_ts, exit_price = ("SL-TRAIL" if cfg.enable_trailing_sl else "SL", ts, dyn_sl)
                break
            if low <= tp:
                exit_reason, exit_ts, exit_price = ("TP", ts, tp)
                break
            if row["RSI"] <= 30.0:
                exit_reason, exit_ts, exit_price = ("RSI-EXIT", ts, close)
                break

        # EOD square-off
        so = _ts_on_day(ts, cfg.squareoff_time, cfg.timezone)
        if ts >= so:
            exit_reason, exit_ts, exit_price = ("EOD", ts, close)
            break

    if exit_ts is None:
        last_ts = after.index[-1]
        exit_reason, exit_ts, exit_price = ("EOD", last_ts, float(after.iloc[-1]["Close"]))

    # P&L and charges
    if direction == "long":
        gross = (exit_price - entry_price) * qty
        buy_turnover  = entry_price * qty
        sell_turnover = exit_price * qty
    else:
        gross = (entry_price - exit_price) * qty
        buy_turnover  = exit_price * qty
        sell_turnover = entry_price * qty

    fees = groww_intraday_charges(buy_turnover, sell_turnover)
    net = gross - fees["total_charges"]

    return {
        "date": entry_ts.date().isoformat(),
        "ticker": ticker,
        "direction": direction,
        "entry_time": entry_ts.isoformat(),
        "entry_price": round(entry_price, 2),
        "qty": qty,
        "sl_price_initial": round(static_sl, 2),
        "tp_price": round(tp, 2),
        "sl_trailing_enabled": cfg.enable_trailing_sl,
        "trail_rupees": trail_per_share,
        "exit_time": exit_ts.isoformat(),
        "exit_price": round(float(exit_price), 2),
        "gross_pnl": round(float(gross), 2),
        "charges": round(float(fees["total_charges"]), 2),
        "net_pnl": round(float(net), 2),
        "exit_reason": exit_reason,
        "buy_turnover": round(float(buy_turnover), 2),
        "sell_turnover": round(float(sell_turnover), 2),
        "cash_capital": round(cfg.capital_per_stock, 2),
        "leverage": cfg.intraday_leverage,
        "notional_at_entry": round(entry_price * qty, 2),
    }

# =========================
# BACKTEST
# =========================
def run_backtest(cfg: Config) -> pd.DataFrame:
    all_trades = []

    for ticker in cfg.tickers:
        log.info(f"Downloading {ticker} ...")
        df = fetch_5m_dataframe(ticker, cfg.start_date, cfg.end_date, cfg.interval, cfg.timezone)
        if df.empty:
            log.warning(f"No data for {ticker}")
            continue

        df = add_indicators(df, cfg)
        dates = sorted(list({pd.Timestamp(ts).date() for ts in df.index}))

        for d in dates:
            day_df = session_filter(df, d, cfg.timezone)
            if len(day_df) < 20:
                continue

            trade_count = 0
            last_exit_ts: Optional[pd.Timestamp] = None

            while trade_count < cfg.max_trades_per_day:
                direction, en_ts, en_price = next_signal_after(day_df, cfg, last_exit_ts)
                if en_ts is None:
                    break

                trade = simulate_trade(day_df, direction, en_ts, en_price, cfg, ticker)
                if not trade:
                    break

                all_trades.append(trade)
                trade_count += 1
                last_exit_ts = pd.Timestamp(trade["exit_time"])

    trades = pd.DataFrame(all_trades)
    if trades.empty:
        print("No trades generated.")
        return trades

    trades.sort_values(by=["date", "ticker", "entry_time"], inplace=True)
    trades.to_csv(cfg.out_file, index=False)
    return trades

# =========================
# METRICS
# =========================
def max_drawdown(series: pd.Series) -> float:
    cum = series.cumsum()
    peak = cum.cummax()
    dd = cum - peak
    return float(dd.min())

def summarize(trades: pd.DataFrame):
    print("\n=== OVERALL METRICS ===")
    n = len(trades)
    if n == 0:
        print("No trades generated.")
        return

    wins = (trades["net_pnl"] > 0).sum()
    win_rate = 100.0 * wins / n
    gross = float(trades["gross_pnl"].sum())
    charges = float(trades["charges"].sum())
    net = float(trades["net_pnl"].sum())
    mdd = max_drawdown(trades["net_pnl"])

    print(f"Trades: {n} | Win rate: {win_rate:.1f}%")
    print(f"Gross P&L: ₹{gross:,.2f} | Charges: ₹{charges:,.2f} | Net P&L: ₹{net:,.2f}")
    print(f"Max Drawdown (net): ₹{mdd:,.2f}")

    print("\n=== BY TICKER ===")
    by_t = trades.groupby("ticker").agg(
        n=("net_pnl","count"),
        wins=("net_pnl", lambda x: (x>0).sum()),
        gross=("gross_pnl","sum"),
        charges=("charges","sum"),
        net=("net_pnl","sum"),
        win_rate=("net_pnl", lambda x: 100.0*(x>0).mean())
    ).reset_index()
    by_t["win_rate"] = by_t["win_rate"].round(1)
    print(by_t.to_string(index=False))

    print("\n=== BY DIRECTION ===")
    by_dir = trades.groupby("direction").agg(
        n=("net_pnl","count"),
        wins=("net_pnl", lambda x: (x>0).sum()),
        gross=("gross_pnl","sum"),
        charges=("charges","sum"),
        net=("net_pnl","sum"),
        win_rate=("net_pnl", lambda x: 100.0*(x>0).mean())
    ).reset_index()
    by_dir["win_rate"] = by_dir["win_rate"].round(1)
    print(by_dir.to_string(index=False))

    print("\nWrote trades to:", CFG.out_file)

# =========================
# MAIN
# =========================
if __name__ == "__main__":
    trades = run_backtest(CFG)
    if not trades.empty:
        summarize(trades)


2025-11-05 00:08:10 | INFO | Downloading ADANIENT.NS ...
2025-11-05 00:08:10 | INFO | Downloading ADANIPORTS.NS ...
2025-11-05 00:08:10 | INFO | Downloading APOLLOHOSP.NS ...
2025-11-05 00:08:10 | INFO | Downloading ASIANPAINT.NS ...
2025-11-05 00:08:10 | INFO | Downloading AXISBANK.NS ...
2025-11-05 00:08:10 | INFO | Downloading BAJAJ-AUTO.NS ...
2025-11-05 00:08:10 | INFO | Downloading BAJFINANCE.NS ...
2025-11-05 00:08:11 | INFO | Downloading BAJAJFINSV.NS ...
2025-11-05 00:08:11 | INFO | Downloading BEL.NS ...
2025-11-05 00:08:11 | INFO | Downloading BHARTIARTL.NS ...
2025-11-05 00:08:11 | INFO | Downloading CIPLA.NS ...
2025-11-05 00:08:11 | INFO | Downloading COALINDIA.NS ...
2025-11-05 00:08:11 | INFO | Downloading DRREDDY.NS ...
2025-11-05 00:08:11 | INFO | Downloading EICHERMOT.NS ...
2025-11-05 00:08:11 | INFO | Downloading ETERNAL.NS ...
2025-11-05 00:08:11 | INFO | Downloading GRASIM.NS ...
2025-11-05 00:08:11 | INFO | Downloading HCLTECH.NS ...
2025-11-05 00:08:11 | INFO |


=== OVERALL METRICS ===
Trades: 230 | Win rate: 40.9%
Gross P&L: ₹23,156.97 | Charges: ₹51,561.12 | Net P&L: ₹-28,404.15
Max Drawdown (net): ₹-46,427.22

=== BY TICKER ===
       ticker  n  wins     gross  charges       net  win_rate
  ADANIENT.NS  5     4  14000.00  1119.55  12880.45      80.0
ADANIPORTS.NS  8     4    -83.08  1794.68  -1877.76      50.0
APOLLOHOSP.NS  3     1     -9.00   669.01   -678.01      33.3
ASIANPAINT.NS 10     4   -492.29  2242.33  -2734.62      40.0
  AXISBANK.NS  4     1  -4579.00   897.27  -5476.27      25.0
BAJAJ-AUTO.NS  3     2   4590.00   668.69   3921.31      66.7
BAJAJFINSV.NS  1     0  -2000.00   224.65  -2224.65       0.0
BAJFINANCE.NS  4     1  -2000.00   897.28  -2897.28      25.0
       BEL.NS  5     1  -5037.50  1123.11  -6160.61      20.0
BHARTIARTL.NS  4     2   1378.08   896.83    481.25      50.0
     CIPLA.NS  8     1 -13234.39  1795.45 -15029.84      12.5
 COALINDIA.NS  5     4   9690.45  1121.89   8568.58      80.0
   DRREDDY.NS  3     